In [1]:
import os, warnings
warnings.simplefilter("ignore")

import torch
from transformers import *
from transformers.utils import logging as hf_logging
from fastai.text.all import *

from blurr.text.data.all import *
from blurr.text.modeling.all import *

In [2]:
seed=1

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'


hf_logging.set_verbosity_error()
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
def get_oversampled_dls(dls, seed=1, baseline_factor=1.0, noise_limit=0.15, label='speaker'):
    df_train = dls.train.items
    max_dialog = df_train[label].value_counts().max()

    dfs_oversampled = [df_train]
    for _, group in df_train.groupby('speaker'):
        rand_mult = random.uniform(baseline_factor-noise_limit, baseline_factor + noise_limit)
        sample_amt_to_max = max_dialog - len(group)
        
        sample_amt = max(int(sample_amt_to_max * rand_mult), len(group))
        
        dfs_oversampled.append(group.sample(sample_amt, replace=True, random_state=seed))
        
    dls.train.items = pd.concat(dfs_oversampled)
    return dls

In [4]:
data_path = Path('../data')
df = pd.read_csv(data_path/'train21_shuffled.csv')
df_test = pd.read_csv(data_path/'test21_shuffled.csv')
df

,season,episode,scene,line_text,speaker,deleted
0,7,24,8,[conducting interview] Your paper experience is very interesting. Do you think you could use that experience to inform decisions here?,Jim,False
1,9,10,27,I'm not gonna lie. Lye!,Dwight,False
2,9,9,27,Take a bowl and pass it down.,Dwight,False
3,3,15,25,It's a miracle.,Dwight,False
4,7,1,1,This is how you build a business. This is how you make it in this country.,Ryan,False
...,...,...,...,...,...,...
44370,3,5,41,I don't know. It felt far.,Pam,False
44371,3,10,43,"Excuse me [tries to take away meat with chopsticks] Ah, un guard. [Fights with chopsticks and laughs] Family style.",Michael,False
44372,5,23,19,I never went to Thailand.,Ryan,False
44373,3,4,1,"[breathless] All right. Okay. [goes back down pretend stairs, crawls on belly to the kitchen for the coffee]",Michael,False


In [5]:
n_labels = len(df['speaker'].unique())

21

In [6]:
pretrained_model_name = 'bert-base-uncased'

'bert-base-uncased'

In [7]:
%%time

noise_limits = [0.1]
batch_sizes = [8, 16]
lrs = [0.003, 0.002, 0.001]
seeds = [1, 23]

epochs=10

for noise in noise_limits:
    for bs in batch_sizes:
        for lr in lrs:
            for seed in seeds:
                learn = BlearnerForSequenceClassification.from_data(
                    df, 
                    pretrained_model_name, 
                    dl_kwargs={"bs": bs, "seed": seed},
                    learner_kwargs={"metrics": accuracy},
                    text_attr='line_text',
                    label_attr='speaker',
                    n_labels = n_labels,
                    dblock_splitter=RandomSplitter(valid_pct=0.1, seed=seed)
                )

                print(f'lr: {lr}, bs: {bs}, noise: {noise}, seed: {seed}')
                learn.dls = get_oversampled_dls(learn.dls, seed=seed, noise_limit=noise)
                test_dl = learn.dls.test_dl(df_test, with_labels='True', label_col='speaker')

                learn.fit_one_cycle(epochs, lr_max=lr)
                
                res=learn.validate(dl=test_dl)
                print(f'Validation results: [cost, accuracy]:{res}')

                learn.export(f'BERT_accuracy{"%.5f"%res[1]}_lr{lr}_bs{bs}_seed{seed}')

lr: 0.003, bs: 8, noise: 0.1, seed: 1


epoch,train_loss,valid_loss,accuracy,time
0,2.539172,2.459905,0.243858,01:58
1,2.465613,2.424700,0.266396,02:00
2,2.505112,2.421263,0.266622,01:58
3,2.466475,2.419780,0.263016,01:56
4,2.451088,2.404349,0.269551,01:57
5,2.375927,2.408864,0.258057,02:01
6,2.369609,2.349365,0.289610,01:57
7,2.325718,2.341902,0.293216,01:58
8,2.358683,2.328231,0.288258,02:02
9,2.212749,2.328589,0.289385,01:59


Validation results: [cost, accuracy]:[2.31400728225708, 0.29468846321105957]
lr: 0.003, bs: 8, noise: 0.1, seed: 23


epoch,train_loss,valid_loss,accuracy,time
0,2.419420,2.463366,0.233266,02:00
1,2.452494,2.468272,0.216362,02:01
2,2.476475,2.427810,0.249493,02:01
3,2.523357,2.454985,0.244084,02:03
4,2.439986,2.403972,0.260762,01:59
5,2.365987,2.383095,0.264142,02:00
6,2.350943,2.372041,0.268650,02:00
7,2.346364,2.351495,0.276764,01:59
8,2.356135,2.348322,0.277890,01:57
9,2.289366,2.348820,0.276313,01:58


Validation results: [cost, accuracy]:[2.3313820362091064, 0.2830694615840912]
lr: 0.002, bs: 8, noise: 0.1, seed: 1


...

Validation results: [cost, accuracy]:[2.301335096359253, 0.2951991856098175]
lr: 0.001, bs: 16, noise: 0.1, seed: 1


epoch,train_loss,valid_loss,accuracy,time
0,2.494608,2.484764,0.248817,01:06
1,2.437532,2.446423,0.253550,01:07
2,2.422692,2.392617,0.267748,01:06
3,2.392332,2.365801,0.275862,01:08
4,2.330584,2.341162,0.292765,01:07
5,2.271879,2.330481,0.288709,01:07
6,2.252472,2.330214,0.290061,01:07
7,2.288999,2.313688,0.296371,01:08
8,2.215034,2.312549,0.299301,01:08
9,2.239938,2.315486,0.297949,01:08


Validation results: [cost, accuracy]:[2.304358720779419, 0.2914964258670807]
lr: 0.001, bs: 16, noise: 0.1, seed: 23


epoch,train_loss,valid_loss,accuracy,time
0,2.449923,2.482248,0.221997,01:06
1,2.445080,2.429351,0.259410,01:06
2,2.386391,2.394469,0.266171,01:07
3,2.381663,2.378314,0.267523,01:07
4,2.342631,2.343393,0.275186,01:08
5,2.293617,2.338420,0.278792,01:08
6,2.304154,2.344872,0.274961,01:08
7,2.253644,2.320167,0.282398,01:08
8,2.232118,2.319192,0.288483,01:08
9,2.205819,2.320465,0.288032,01:07


Validation results: [cost, accuracy]:[2.3020877838134766, 0.29009193181991577]
CPU times: user 3h 3min 13s, sys: 5min 45s, total: 3h 8min 59s
Wall time: 3h 12min 59s
